# Default OpenAI approach

In [6]:
import os
import sqlite3
from pydantic import BaseModel
from openai import OpenAI

from dotenv import load_dotenv

load_dotenv()

True

In [7]:
import phoenix as px
from phoenix.otel import register

# pip install -q openinference-instrumentation-openai
from openinference.instrumentation.openai import OpenAIInstrumentor

px.launch_app()

tracer_provider = register(
    project_name="my-llm-app",  # Default is 'default'
    endpoint="http://localhost:4317",  # Sends traces using gRPC
)

OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
OpenTelemetry Tracing Details
|  Phoenix Project: my-llm-app
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [ ]:
# Build system prompt
with open("prompts/system_prompt.md", "r") as file:
    system_prompt = file.read()

# system_prompt = system_prompt.replace("<<INSERT DATABASE INFORMATION>>", db_structure)

print(system_prompt[:200])

In [ ]:
client = OpenAI()

class Issue(BaseModel):
    type: str
    explanation: str
    recommendation: str


class SQLOutput(BaseModel):
    chain_of_thought: str
    issues: list[Issue]
    sql_query: str

    def __str__(self):
        return (
            f"SQLOutput(\n"
            f"  chain_of_thought: \"{self.chain_of_thought}\",\n"
            f"  issues: {self.issues},\n"
            f"  sql_query: \"{self.sql_query}\"\n"
            f")"
        )


def get_response(user_query: str):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        # model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_query},
        ],
        response_format=SQLOutput,
        temperature=0
    )
    usage = completion.usage
    response = completion.choices[0].message.parsed
    if response and response.sql_query != "NO_QUERY":
        try:
            query_result = run_sql_query(response.sql_query)
        except Exception as e:
            print(e)
            return response, "ERROR" + e, usage
        return response, query_result, usage
    return response, "NO_QUERY", usage

## Evaluation

In [ ]:
import json

with open("questions.json", "r") as file:
    questions = json.load(file)

In [ ]:
# Manual evaluation
